# MILP vs GNN Benchmark Overview

Run the cells below to compare MILP baseline estimates with the selected GNN model. Update RUN_DIR if you want to analyse a different experiment.

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

def _locate_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        candidate = candidate.resolve()
        if (candidate / 'src').exists():
            return candidate
    raise RuntimeError('Could not locate project root containing "src" directory')

PROJECT_ROOT = _locate_project_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

os.chdir(PROJECT_ROOT)

try:
    import seaborn as sns
    sns.set_theme(style='whitegrid')
except ImportError:  # pragma: no cover
    sns = None
    plt.style.use('ggplot')

from src.analysis.performance import build_combined_performance

RUN_DIR = Path('outputs/gnn_runs/baseline')
SPLIT = 'test'
combined_df, summary = build_combined_performance(RUN_DIR, split=SPLIT, device='cpu')
if combined_df.empty:
    raise ValueError(f"No evaluation rows found for split '{SPLIT}'.")
display(pd.DataFrame([summary]))
display(combined_df[['scenario_id', 'milp_objective', 'gnn_cost_gap', 'gnn_violation_rate', 'gnn_speedup_est']])


FileNotFoundError: [Errno 2] No such file or directory: 'outputs\\gnn_runs\\baseline\\training_config_resolved.yaml'

## Cost Gap by Scenario

Relative cost gap compares the weighted dispatch cost produced by the GNN to the MILP objective (0 indicates parity).

In [1]:
sorted_df = combined_df.sort_values('gnn_cost_gap').reset_index(drop=True)
labels = sorted_df['scenario_id'].astype(str)
x = range(len(sorted_df))
fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(x, sorted_df['gnn_cost_gap'], color='#4c72b0')
ax.axhline(0, color='black', linestyle='--', linewidth=1, label='MILP objective')
ax.set_xticks(list(x))
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_ylabel('Relative cost gap')
ax.set_xlabel('Scenario')
ax.set_title('GNN cost gap relative to MILP objective')
ax.legend()
plt.tight_layout()


NameError: name 'combined_df' is not defined

## Feasibility / Violation Rate

Constraint violation rate corresponds to the share of node-time points with unmet demand (after decoder corrections).

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(x, sorted_df['gnn_violation_rate'], color='#dd8452')
ax.set_xticks(list(x))
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_ylabel('Violation rate')
ax.set_xlabel('Scenario')
ax.set_ylim(0, 1)
ax.set_title('Feasibility gap (lower is better)')
plt.tight_layout()


## Estimated Speed-up

MILP runtime uses the estimated CPU hours field from the scenario metadata, while GNN time comes from measured inference wall-clock.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
speedups = sorted_df['gnn_speedup_est']
ax.bar(x, speedups, color='#55a868')
ax.set_xticks(list(x))
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_yscale('log')
ax.set_ylabel('Estimated MILP / GNN speed-up')
ax.set_xlabel('Scenario')
ax.set_title('Runtime advantage (log scale)')
plt.tight_layout()


## Cost Gap vs Feasibility Scatter

Use the scatter plot to see whether cost accuracy trades off with feasibility.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(combined_df['gnn_cost_gap'], combined_df['gnn_violation_rate'], s=80, color='#4c72b0')
for _, row in combined_df.iterrows():
    ax.annotate(row['scenario_id'][-6:], (row['gnn_cost_gap'], row['gnn_violation_rate']),
                textcoords='offset points', xytext=(4, 4), fontsize=9)
ax.axhline(0, color='grey', linewidth=0.8)
ax.axvline(0, color='grey', linewidth=0.8)
ax.set_xlabel('Relative cost gap')
ax.set_ylabel('Violation rate')
ax.set_title('Cost accuracy vs feasibility')
plt.tight_layout()
